In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [2]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
# Load and modify VGG16 model
class ModifiedVGG16(nn.Module):
    def __init__(self, num_classes=10):
        super(ModifiedVGG16, self).__init__()
        # Load pretrained VGG16
        vgg16 = torchvision.models.vgg16(pretrained=True)
        
        # Keep all layers except the classifier
        self.features = vgg16.features
        
        # Freeze the feature layers
        for param in self.features.parameters():
            param.requires_grad = False
            
        # Create new classifier layers
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 1024),  # Modified layer
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(1024, num_classes)  # Modified layer
        )
                
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [3]:
# Hyperparameters
num_epochs = 10
batch_size = 32
learning_rate = 0.001


In [4]:
# Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # VGG16 expects 224x224 images
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = torchvision.datasets.CIFAR10(
    root='./data', 
    train=True,
    download=True, 
    transform=transform
)

test_dataset = torchvision.datasets.CIFAR10(
    root='./data', 
    train=False,
    download=True, 
    transform=transform
)

train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False
)


Files already downloaded and verified
Files already downloaded and verified


In [10]:

# Initialize the model
model = ModifiedVGG16(num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=learning_rate)


C:\Users\Besose\anaconda3\envs\tf\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Besose\anaconda3\envs\tf\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\Besose/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100%|███████████████████████████████████████████████████████████████████████████████| 528M/528M [02:36<00:00, 3.53MB/s]


In [11]:
# Training loop
def train_model():
    print("Starting training...")
    model.train()
    total_step = len(train_loader)
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # Calculate accuracy
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            running_loss += loss.item()
            
            if (i + 1) % 100 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], '
                      f'Loss: {loss.item():.4f}, '
                      f'Accuracy: {100 * correct / total:.2f}%')
        
        print(f'Epoch [{epoch+1}/{num_epochs}] completed. '
              f'Average Loss: {running_loss/total_step:.4f}, '
              f'Final Accuracy: {100 * correct / total:.2f}%')


In [13]:
# Train the model
train_model()

Starting training...
Epoch [1/10], Step [100/1563], Loss: 1.0772, Accuracy: 61.12%
Epoch [1/10], Step [200/1563], Loss: 0.6882, Accuracy: 68.08%
Epoch [1/10], Step [300/1563], Loss: 0.7614, Accuracy: 70.56%
Epoch [1/10], Step [400/1563], Loss: 1.1611, Accuracy: 71.96%
Epoch [1/10], Step [500/1563], Loss: 0.6012, Accuracy: 72.95%
Epoch [1/10], Step [600/1563], Loss: 0.5718, Accuracy: 73.74%
Epoch [1/10], Step [700/1563], Loss: 0.6390, Accuracy: 74.28%
Epoch [1/10], Step [800/1563], Loss: 1.1977, Accuracy: 74.74%
Epoch [1/10], Step [900/1563], Loss: 0.7070, Accuracy: 75.17%
Epoch [1/10], Step [1000/1563], Loss: 0.4895, Accuracy: 75.50%
Epoch [1/10], Step [1100/1563], Loss: 0.5685, Accuracy: 75.81%
Epoch [1/10], Step [1200/1563], Loss: 0.5921, Accuracy: 76.11%
Epoch [1/10], Step [1300/1563], Loss: 0.7950, Accuracy: 76.42%
Epoch [1/10], Step [1400/1563], Loss: 0.4360, Accuracy: 76.62%
Epoch [1/10], Step [1500/1563], Loss: 0.5423, Accuracy: 76.86%
Epoch [1/10] completed. Average Loss: 0.725

In [14]:
# Evaluation function
def evaluate_model():
    print("\nEvaluating model on test set...")
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        print(f'Test Accuracy: {100 * correct / total:.2f}%')


In [15]:
# Evaluate the model
evaluate_model()


Evaluating model on test set...
Test Accuracy: 87.05%


In [16]:
# Save the model
print("\nSaving the model...")
save_path = 'modified_vgg16_cifar10.pth'
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'num_classes': 10
}, save_path)
print(f"Model saved to {save_path}")
    


Saving the model...
Model saved to modified_vgg16_cifar10.pth
